## Anomaly Detection Project Implementation

In [1]:
import os
import sys
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from scipy.io import loadmat   # mat file reading

## import pyod packages and the methods

In [2]:
from pyod.models.pca import PCA
from pyod.models.mcd import MCD
from pyod.models.ocsvm import OCSVM
from pyod.models.lof import LOF
from pyod.models.cblof import CBLOF
from pyod.models.knn import KNN
from pyod.models.hbos import HBOS
from pyod.models.abod import ABOD
from pyod.models.iforest import IForest
from pyod.models.feature_bagging import FeatureBagging

C:\ProgramData\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:143: FutureWarning: The sklearn.utils.testing module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.utils. Anything that cannot be imported from sklearn.utils is now part of the private API.
  warnings.warn(message, FutureWarning)


## Import Metrics Packages

In [3]:
from pyod.utils.utility import standardizer
from pyod.utils.utility import precision_n_scores  # measure to identify perfomance of the model
from sklearn.metrics import roc_auc_score  

## load and define data file read X and Y

In [17]:
## Create list
mat_file_list = ['arrhythmia.mat','cardio.mat','glass.mat','ionosphere.mat','letter.mat','lympho.mat','mnist.mat','musk.mat','optdigits.mat','pendigits.mat','pima.mat','satimage-2.mat','shuttle.mat','vertebral.mat','vowels.mat','wbc.mat']

In [18]:
mat_file_list

['arrhythmia.mat',
 'cardio.mat',
 'glass.mat',
 'ionosphere.mat',
 'letter.mat',
 'lympho.mat',
 'mnist.mat',
 'musk.mat',
 'optdigits.mat',
 'pendigits.mat',
 'pima.mat',
 'satimage-2.mat',
 'shuttle.mat',
 'vertebral.mat',
 'vowels.mat',
 'wbc.mat']

In [19]:
data = loadmat("D://ML//dataset//Anamoly_detec_data/cardio.mat")  #  load dataset mat file

In [20]:
data

{'__header__': b'MATLAB 5.0 MAT-file, written by Octave 3.8.0, 2014-12-18 10:48:09 UTC',
 '__version__': '1.0',
 '__globals__': [],
 'X': array([[ 0.00491231,  0.69319077, -0.20364049, ...,  0.23149795,
         -0.28978574, -0.49329397],
        [ 0.11072935, -0.07990259, -0.20364049, ...,  0.09356344,
         -0.25638541, -0.49329397],
        [ 0.21654639, -0.27244466, -0.20364049, ...,  0.02459619,
         -0.25638541,  1.14001753],
        ...,
        [-0.41835583, -0.91998844, -0.16463485, ..., -1.49268341,
          0.24461959, -0.49329397],
        [-0.41835583, -0.91998844, -0.15093411, ..., -1.42371616,
          0.14441859, -0.49329397],
        [-0.41835583, -0.91998844, -0.20364049, ..., -1.28578165,
          3.58465295, -0.49329397]]),
 'y': array([[0.],
        [0.],
        [0.],
        ...,
        [1.],
        [1.],
        [1.]])}

In [21]:
len(data)

5

In [22]:
data.keys()   #  key name 

dict_keys(['__header__', '__version__', '__globals__', 'X', 'y'])

In [23]:
data.values()  # values of data

dict_values([b'MATLAB 5.0 MAT-file, written by Octave 3.8.0, 2014-12-18 10:48:09 UTC', '1.0', [], array([[ 0.00491231,  0.69319077, -0.20364049, ...,  0.23149795,
        -0.28978574, -0.49329397],
       [ 0.11072935, -0.07990259, -0.20364049, ...,  0.09356344,
        -0.25638541, -0.49329397],
       [ 0.21654639, -0.27244466, -0.20364049, ...,  0.02459619,
        -0.25638541,  1.14001753],
       ...,
       [-0.41835583, -0.91998844, -0.16463485, ..., -1.49268341,
         0.24461959, -0.49329397],
       [-0.41835583, -0.91998844, -0.15093411, ..., -1.42371616,
         0.14441859, -0.49329397],
       [-0.41835583, -0.91998844, -0.20364049, ..., -1.28578165,
         3.58465295, -0.49329397]]), array([[0.],
       [0.],
       [0.],
       ...,
       [1.],
       [1.],
       [1.]])])

In [24]:
type(data['X']),data['X'].shape   # shape of the independent feature

(numpy.ndarray, (1831, 21))

In [25]:
type(data['y']),data['y'].shape     # shape of the dependent features

(numpy.ndarray, (1831, 1))

## Define nine outliers detection tools to be compared

In [26]:
df_columns = ['Data','#Samples','# Dimensions','Outlier Perc','ABOD','CBLOF','FB','HBOS','IForest','KNN','LOF','MCD','OCSVM','PCA']

In [27]:
roc_df = pd.DataFrame(columns =df_columns)  # to create  empty dataset based on columns name
roc_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA


In [28]:
prn_df = pd.DataFrame(columns=df_columns)   # precision_n_score measure to identify perfomance of the model
prn_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA


In [29]:
time_df = pd.DataFrame(columns=df_columns)   # time dataframe
time_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA


## Exploraing All mat file

In [41]:
from time import time
random_state = np.random.RandomState(42)
for mat_file in mat_file_list:
    print("\n...Processing",mat_file,'...')   
    mat = loadmat(os.path.join('D://ML//dataset//Anamoly_detec_data',mat_file))
    
    X = mat['X']
    y = mat['y'].ravel()   # # convert 2d to 1d 
    outliers_fraction = np.count_nonzero(y) / len(y)
    outliers_percentage = round(outliers_fraction * 100,ndigits=4)
    # construct containers for saving results
    roc_list = [mat_file[:-4],X.shape[0],X.shape[1],outliers_percentage]
    prn_list = [mat_file[:-4],X.shape[0],X.shape[1],outliers_percentage]
    time_list = [mat_file[:-4],X.shape[0],X.shape[1],outliers_percentage]
    
    # train test split
    X_train,X_test,y_train,y_test = train_test_split(X,y,test_size=0.4,random_state=random_state)
    
    # standardizating data for processiing
    X_train_norm,X_test_norm = standardizer(X_train,X_test)
    
    classifiers = {'Angle-based outlier Detector(ABOD)':ABOD(contamination=outliers_fraction),
                   'Cluster-based Local Outlier Factore':CBLOF(contamination=outliers_fraction,check_estimator=False,random_state=random_state),
                   'Feature bagging':FeatureBagging(contamination=outliers_fraction,random_state=random_state),
                   'Histogram-based Outliers Detection(HBOS)':HBOS(contamination=outliers_fraction),
                   'Isolation Foreset':IForest(contamination=outliers_fraction,random_state=random_state),
                   'K Nearest Neighbours (KNN)':KNN(contamination=outliers_fraction),
                   'Local Outliers Factore(LOF)':LOF(contamination=outliers_fraction),
                   'Minimum Covariance Determinant (MCD)': MCD(contamination=outliers_fraction,random_state=random_state),
                   'One-class SVM (OCSVM)':OCSVM(contamination=outliers_fraction),
                   'Principal Component Analysis (PCA)': PCA(contamination=outliers_fraction,random_state=random_state)
                  }
    
    for clf_name, clf in classifiers.items():
        t0 = time()
        clf.fit(X_train_norm)
        test_scores = clf.decision_function(X_test_norm)
        t1 = time()
        duration = round(t1-t0,ndigits =4)
        time_list.append(duration)
        
        roc = round(roc_auc_score(y_test,test_scores),ndigits=4)
        prn = round(precision_n_scores(y_test,test_scores),ndigits=4)
        
        print('{clf_name} ROC:{roc},precision @ rank n:{prn},'
             'execution time:{duration}s'.format(
                 clf_name = clf_name,roc=roc,prn=prn,duration=duration))
    
        roc_list.append(roc)
        prn_list.append(prn)
    

temp_df = pd.DataFrame(time_list).transpose()
temp_df.columns = df_columns
time_df = pd.concat([time_df,temp_df],axis =0)

temp_df = pd.DataFrame(roc_list).transpose()
temp_df.columns = df_columns
roc_df = pd.concat([roc_df,temp_df],axis=0)

temp_df = pd.DataFrame(prn_list).transpose()
temp_df.columns = df_columns
prn_df = pd.concat([prn_df,temp_df],axis=0)
    


...Processing arrhythmia.mat ...
Angle-based outlier Detector(ABOD) ROC:0.7687,precision @ rank n:0.3571,execution time:0.254s
Cluster-based Local Outlier Factore ROC:0.7684,precision @ rank n:0.4643,execution time:0.1629s


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:974: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  " removed in 0.25.", FutureWarning)


Feature bagging ROC:0.7799,precision @ rank n:0.5,execution time:0.6846s
Histogram-based Outliers Detection(HBOS) ROC:0.8511,precision @ rank n:0.5714,execution time:0.1109s
Isolation Foreset ROC:0.8478,precision @ rank n:0.5357,execution time:0.6256s
K Nearest Neighbours (KNN) ROC:0.782,precision @ rank n:0.5,execution time:0.0982s
Local Outliers Factore(LOF) ROC:0.7787,precision @ rank n:0.4643,execution time:0.077s


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\covariance\_robust_covariance.py:647: UserWarning: The covariance matrix associated to your dataset is not full rank
  warnings.warn("The covariance matrix associated to your dataset "


Minimum Covariance Determinant (MCD) ROC:0.8228,precision @ rank n:0.4286,execution time:0.9014s
One-class SVM (OCSVM) ROC:0.7986,precision @ rank n:0.5,execution time:0.048s
Principal Component Analysis (PCA) ROC:0.8,precision @ rank n:0.5,execution time:0.067s

...Processing cardio.mat ...
Angle-based outlier Detector(ABOD) ROC:0.5763,precision @ rank n:0.1875,execution time:0.7216s
Cluster-based Local Outlier Factore ROC:0.8221,precision @ rank n:0.4844,execution time:0.1929s

...

Cluster-based Local Outlier Factore ROC:0.3226,precision @ rank n:0.0,execution time:0.079s
Feature bagging ROC:0.2841,precision @ rank n:0.0,execution time:0.048s
Histogram-based Outliers Detection(HBOS) ROC:0.297,precision @ rank n:0.0,execution time:0.003s
Isolation Foreset ROC:0.2279,precision @ rank n:0.0,execution time:0.4947s
K Nearest Neighbours (KNN) ROC:0.2279,precision @ rank n:0.0,execution time:0.016s
Local Outliers Factore(LOF) ROC:0.2713,precision @ rank n:0.0,execution time:0.005s
Minimum Covariance Determinant (MCD) ROC:0.3018,precision @ rank n:0.0,execution time:0.0628s
One-class SVM (OCSVM) ROC:0.3852,precision @ rank n:0.0,execution time:0.002s
Principal Component Analysis (PCA) ROC:0.3917,precision @ rank n:0.0,execution time:0.003s

...Processing vowels.mat ...
Angle-based outlier Detector(ABOD) ROC:0.9332,precision @ rank n:0.4762,execution time:0.5237s
Cluster-based Local Outlier Factore ROC:0.8609,precision @ rank n:0.1429,execution time:0.1119s


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:974: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  " removed in 0.25.", FutureWarning)


Feature bagging ROC:0.9353,precision @ rank n:0.2381,execution time:0.3758s
Histogram-based Outliers Detection(HBOS) ROC:0.6634,precision @ rank n:0.1429,execution time:0.007s
Isolation Foreset ROC:0.7132,precision @ rank n:0.1905,execution time:0.5397s
K Nearest Neighbours (KNN) ROC:0.9592,precision @ rank n:0.4286,execution time:0.1149s
Local Outliers Factore(LOF) ROC:0.9182,precision @ rank n:0.1905,execution time:0.043s
Minimum Covariance Determinant (MCD) ROC:0.6453,precision @ rank n:0.0,execution time:1.1132s
One-class SVM (OCSVM) ROC:0.7189,precision @ rank n:0.1905,execution time:0.052s
Principal Component Analysis (PCA) ROC:0.5703,precision @ rank n:0.1429,execution time:0.004s

...Processing wbc.mat ...
Angle-based outlier Detector(ABOD) ROC:0.9083,precision @ rank n:0.3333,execution time:0.1349s
Cluster-based Local Outlier Factore ROC:0.934,precision @ rank n:0.5556,execution time:0.0859s
Feature bagging ROC:0.9557,precision @ rank n:0.6667,execution time:0.0899s


C:\ProgramData\Anaconda3\lib\site-packages\sklearn\cluster\_kmeans.py:974: FutureWarning: 'n_jobs' was deprecated in version 0.23 and will be removed in 0.25.
  " removed in 0.25.", FutureWarning)


Histogram-based Outliers Detection(HBOS) ROC:0.9526,precision @ rank n:0.6667,execution time:0.014s
Isolation Foreset ROC:0.9068,precision @ rank n:0.5556,execution time:0.5037s
K Nearest Neighbours (KNN) ROC:0.9557,precision @ rank n:0.6667,execution time:0.029s
Local Outliers Factore(LOF) ROC:0.9643,precision @ rank n:0.6667,execution time:0.009s
Minimum Covariance Determinant (MCD) ROC:0.9037,precision @ rank n:0.3333,execution time:0.0849s
One-class SVM (OCSVM) ROC:0.9479,precision @ rank n:0.6667,execution time:0.007s
Principal Component Analysis (PCA) ROC:0.9176,precision @ rank n:0.5556,execution time:0.004s


In [45]:
time_df

,Data,#Samples,# Dimensions,Outlier Perc,ABOD,CBLOF,FB,HBOS,IForest,KNN,LOF,MCD,OCSVM,PCA
0,wbc,378,30,5.5556,0.1939,0.1069,0.2449,0.041,0.8305,0.03,0.01,0.0969,0.007,0.004
0,wbc,378,30,5.5556,0.1349,0.0859,0.0899,0.014,0.5037,0.029,0.009,0.0849,0.007,0.004
